In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/next-concept-predictor/venv/lib/python3.11/site-packages')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [104]:
# setting hyperparameters
b=2
n=7
d=3
v=12

In [105]:
E_dict = {" I":0,
          " think":1,
         " there":2,
         "fore":3,
         " am":4,
         "Every":5,
         " cloud":6,
         " has":7,
         " a":8,
         " silver":9,
         " lining":10,
         "<endoftext>":11}
E_dict

{' I': 0,
 ' think': 1,
 ' there': 2,
 'fore': 3,
 ' am': 4,
 'Every': 5,
 ' cloud': 6,
 ' has': 7,
 ' a': 8,
 ' silver': 9,
 ' lining': 10,
 '<endoftext>': 11}

In [106]:
# create sequence of tokens
S_full_text = [[' I', ' think', ' there', 'fore', ' I', ' am', '<endoftext>'],
               ['Every', ' cloud', ' has', ' a', ' silver', ' lining', '<endoftext>']]
# turn into indices
S_full_indices = [[E_dict[word] for word in sentence] for sentence in S_full_text]
# turn into a tensor
S_full = torch.tensor(S_full_indices)
print("S_full: ", S_full.dtype, S_full.shape, S_full)

# starting off with the first token for each sequence
i=0
if i==0: 
    S_i = S_full[:,i].unsqueeze(dim=1) 
else: 
    S_i = S_full[:,i]
print("S_i: ", S_i.dtype, S_i.shape, S_i)

S_full:  torch.int64 torch.Size([2, 7]) tensor([[ 0,  1,  2,  3,  0,  4, 11],
        [ 5,  6,  7,  8,  9, 10, 11]])
S_i:  torch.int64 torch.Size([2, 1]) tensor([[0],
        [5]])


In [107]:
# embedding matrix
#torch.manual_seed(420)
E = torch.randn(v,d)

layer_norm = nn.LayerNorm(d)

# Apply layer normalization to the matrix
E = layer_norm(E)
print("E: ", E.shape, E)

# Look up the embeddings
X = F.embedding(S_i, E)
print("X: ", X.shape, X)

E:  torch.Size([12, 3]) tensor([[-0.6706, -0.7430,  1.4136],
        [-1.2875,  0.1371,  1.1504],
        [ 1.3741, -0.9765, -0.3976],
        [-0.1505,  1.2930, -1.1426],
        [ 1.3894, -0.9232, -0.4662],
        [-1.4127,  0.6497,  0.7630],
        [ 1.3712, -0.3858, -0.9854],
        [ 0.2996, -1.3467,  1.0471],
        [-1.3994,  0.5228,  0.8766],
        [ 1.3821, -0.4315, -0.9506],
        [-0.8486,  1.4040, -0.5554],
        [ 0.5919, -1.4083,  0.8163]], grad_fn=<NativeLayerNormBackward0>)
X:  torch.Size([2, 1, 3]) tensor([[[-0.6706, -0.7430,  1.4136]],

        [[-1.4127,  0.6497,  0.7630]]], grad_fn=<EmbeddingBackward0>)


In [108]:
W1 = torch.randn(d,32)
X1 = torch.matmul(X,W1)
print("X1: ", X1.shape, X1)
relu = nn.ReLU()
X2 = relu(X1)
W2 = torch.randn(32,d)
Xf = torch.matmul(X2,W2)
print("Xf: ", Xf.shape, Xf)

X1:  torch.Size([2, 1, 32]) tensor([[[-0.6582,  1.8389,  1.0779,  1.1877, -0.9398, -0.5339,  2.3935,
          -0.6338,  2.3696,  3.3814, -0.5339,  0.1286,  0.9073,  0.3305,
          -1.1010,  1.4807, -0.6096, -1.1420, -1.8985,  2.0291, -1.1432,
          -1.3912,  0.5508, -0.5454, -1.0226, -0.4329,  0.7349,  0.1241,
           0.4398, -0.9780, -1.3265,  1.4699]],

        [[ 0.0551,  1.0327, -0.6585, -0.6271,  2.0542, -0.0442,  3.1309,
          -2.2739, -0.5902,  0.2608, -1.4753, -0.8457,  3.6719,  1.2442,
          -3.3978, -1.0615, -2.1704, -3.5118, -1.4153,  0.5634,  0.0936,
           0.5537,  2.3109,  1.6832, -1.1188, -3.1995,  0.0390,  3.3204,
          -2.1421, -0.5640,  1.4787,  2.4649]]], grad_fn=<UnsafeViewBackward0>)
Xf:  torch.Size([2, 1, 3]) tensor([[[  1.9966,  -4.3303,  -4.1194]],

        [[ -3.0511, -11.4721,   0.9794]]], grad_fn=<UnsafeViewBackward0>)


In [109]:
# select final row
Y = Xf[:,-1:].squeeze(dim=1)
print("Y: ", Y.shape, Y)

Y:  torch.Size([2, 3]) tensor([[  1.9966,  -4.3303,  -4.1194],
        [ -3.0511, -11.4721,   0.9794]], grad_fn=<SqueezeBackward1>)


In [110]:
Z = torch.matmul(Y,E.T)
print("Z: ", Z.shape, Z)

Z:  torch.Size([2, 12]) tensor([[ -3.9449,  -7.9033,   8.6102,  -1.1929,   8.6921,  -8.7770,   8.4677,
           2.1163,  -8.6689,   8.5437,  -5.4860,   3.9173],
        [ 11.9540,   3.4828,   6.6205, -15.4935,   5.8950,  -2.3953,  -0.7225,
          15.5612,  -0.8690,  -0.1979, -14.0619,  15.1491]],
       grad_fn=<MmBackward0>)


In [111]:
# Normalize each vector to have a unit length
E_norms = torch.norm(E, p=2, dim=1, keepdim=True)
E_unit = E / E_norms
print("E_norms: ", E_norms.shape, E_norms)
print("E_unit: ", E_unit.shape, E_unit)

X_norms = torch.norm (X, p=2, dim=2, keepdim=True)
X_unit = X / X_norms
print("X_norms: ", X_norms.shape, X_norms)
print("X_unit: ", X_unit.shape, X_unit)

E_norms:  torch.Size([12, 1]) tensor([[1.7320],
        [1.7320],
        [1.7320],
        [1.7320],
        [1.7320],
        [1.7320],
        [1.7320],
        [1.7320],
        [1.7320],
        [1.7320],
        [1.7320],
        [1.7320]], grad_fn=<LinalgVectorNormBackward0>)
E_unit:  torch.Size([12, 3]) tensor([[-0.3872, -0.4290,  0.8161],
        [-0.7433,  0.0791,  0.6642],
        [ 0.7934, -0.5638, -0.2296],
        [-0.0869,  0.7465, -0.6597],
        [ 0.8022, -0.5330, -0.2692],
        [-0.8156,  0.3751,  0.4405],
        [ 0.7917, -0.2228, -0.5689],
        [ 0.1730, -0.7775,  0.6046],
        [-0.8079,  0.3018,  0.5061],
        [ 0.7980, -0.2491, -0.5488],
        [-0.4899,  0.8106, -0.3207],
        [ 0.3418, -0.8131,  0.4713]], grad_fn=<DivBackward0>)
X_norms:  torch.Size([2, 1, 1]) tensor([[[1.7320]],

        [[1.7320]]], grad_fn=<LinalgVectorNormBackward0>)
X_unit:  torch.Size([2, 1, 3]) tensor([[[-0.3872, -0.4290,  0.8161]],

        [[-0.8156,  0.3751,  0.4405]

In [112]:
# select final row
Y_unit = X_unit[:,-1:].squeeze(dim=1)
print("Y_unit: ", Y_unit.shape, Y_unit)

Y_unit:  torch.Size([2, 3]) tensor([[-0.3872, -0.4290,  0.8161],
        [-0.8156,  0.3751,  0.4405]], grad_fn=<SqueezeBackward1>)


In [113]:
Z_unit = torch.matmul(Y_unit,E_unit.T)
print("Z_unit: ", Z_unit.shape, Z_unit)

Z_unit:  torch.Size([2, 12]) tensor([[ 1.0000,  0.7960, -0.2527, -0.8250, -0.3016,  0.5144, -0.6753,  0.7600,
          0.5964, -0.6500, -0.4197,  0.6011],
        [ 0.5144,  0.9286, -0.9597,  0.0603, -0.9728,  1.0000, -0.9799, -0.1664,
          0.9951, -0.9860,  0.5624, -0.3761]], grad_fn=<MmBackward0>)


In [114]:
layer_norm_v = nn.LayerNorm(v)

Z_standard = layer_norm_v(Z)
print("Z_standard: ", Z_standard.shape, Z_standard)

Z_standard:  torch.Size([2, 12]) tensor([[-0.6229, -1.1951,  1.1919, -0.2251,  1.2037, -1.3214,  1.1713,  0.2532,
         -1.3057,  1.1823, -0.8457,  0.5135],
        [ 1.0324,  0.1470,  0.4749, -1.8366,  0.3991, -0.4675, -0.2926,  1.4095,
         -0.3079, -0.2378, -1.6869,  1.3664]],
       grad_fn=<NativeLayerNormBackward0>)
